In [150]:
import pandas as pd
from src.cleaning import LandCleaner
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [151]:
land = pd.read_csv('output/listing_details.csv')
land['des + title'] = 'Title: '+ land['title'] + '. Description: ' + land['description']
land.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1981 entries, 0 to 1980
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   url            1981 non-null   object 
 1   title          1981 non-null   object 
 2   id             1981 non-null   int64  
 3   latitude       1980 non-null   float64
 4   longitude      1980 non-null   float64
 5   short_address  1981 non-null   object 
 6   address_parts  1981 non-null   object 
 7   main_info      1981 non-null   object 
 8   description    1981 non-null   object 
 9   other_info     1981 non-null   object 
 10  image_urls     1981 non-null   object 
 11  des + title    1981 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 185.8+ KB


In [167]:
land_purpose = ['đất trồng lúa|đất chuyên trồng lúa|đất lúa|đất (?:\S+\s){0,2}trồng cây hàng năm|đất (?:\S+\s){0,2}trồng cây hằng năm|(?:đất\s)?\W(?:luc|lua|lun|luk|lun|bhk|nhk|hnk|nkh)\W', # Đất trồng cây hằng năm
                'đất (?:trồng\s)?cây lâu năm|đất cln|cây lâu năm|(?:đất\s)?(?:lnc|lnq|lnk)', # Đất trồng cây lâu năm
                'đất trồng cây (?!hàng năm|hằng năm|lâu năm)|đất vườn', # Đất vườn
                'đất rừng|(?:đất\s)?\W(?:rsx|rph|rdd|rsn|rst|rsm|rsk|rsm|rpn|rpt|rpk|rpm|rdn|rdt|rdk|rdm)\W', # Đất rừng
                'đất nông nghiệp khác|đất nkh', # Đất nông nghiệp khác
                'đất ở|thổ cư|full thổ|(?:đất\s)?\W(?:ont|odt)\W', # Đất ở
                'đất khu công nghiệp|đất công nghiệp|đất kcn|đất (?:skk|skn|skt)', # Đất khu công nghiệp
                'đất thương mại|đất tm\W|đất dịch vụ|đất dvu\W|đất dv\W|đất tmdv|đất kinh doanh|(?:đất\s)?\Wtmd\W', # Đất thương mại dịch vụ
               ]
land_purpose_dict = {'đất trồng lúa|đất chuyên trồng lúa|đất lúa|đất (?:\S+\s){0,2}trồng cây hàng năm|đất (?:\S+\s){0,2}trồng cây hằng năm|(?:đất\s)?\W(?:luc|lua|lun|luk|lun|bhk|nhk|hnk|nkh)\W': 'Đất trồng cây hàng năm', # Đất trồng cây hằng năm
                'đất (?:trồng\s)?cây lâu năm|đất cln|cây lâu năm|(?:đất\s)?(?:lnc|lnq|lnk)': 'Đất trồng cây lâu năm', # Đất trồng cây lâu năm
                'đất trồng cây\W(?!hàng năm|hằng năm|lâu năm)|đất vườn': 'Đất vườn', # Đất vườn
                'đất rừng|(?:đất\s)?\W(?:rsx|rph|rdd|rsn|rst|rsm|rsk|rsm|rpn|rpt|rpk|rpm|rdn|rdt|rdk|rdm)\W': 'Đất rừng', # Đất rừng
                'đất nông nghiệp khác|đất nkh': 'Đất nông nghiệp khác', # Đất nông nghiệp khác
                'đất ở|thổ cư|full thổ|(?:đất\s)?\W(?:ont|odt)\W': 'Đất ở', # Đất ở
                'đất khu công nghiệp|đất kcn|đất (?:skk|skn|skt)': 'Đất khu công nghiệp', # Đất khu công nghiệp
                'đất thương mại|đất tm\W|đất dịch vụ|đất dvu\W|đất dv\W|đất tmdv|đất kinh doanh|(?:đất\s)?\Wtmd\W': 'Đất thương mại, dịch vụ', # Đất thương mại dịch vụ
                }

# def search_purpose(des_tit):
#     num_pur = []
#     des_tit = des_tit.lower()
#     for pur in land_purpose:
#         if pur in des_tit:
#             num_pur.append(pur)
    
#     if len(num_pur) > 1:
#         return f'Đất hỗn hợp: {num_pur}'
#     if len(num_pur) == 1:
#         return num_pur
#     return 'Đất ở'

def search_purpose(des_tit):
    num_pur = []
    des_tit = des_tit.lower()

    for pur in land_purpose_dict.keys():
        search_for_pur = re.search(pur, des_tit)
        if search_for_pur:
            num_pur.append(land_purpose_dict[pur])

    if len(num_pur) > 1:
        return f'Đất hỗn hợp: {num_pur}'
    if len(num_pur) == 1:
        return f'{num_pur}'
    return 'Đất ở'

def new_search_purpose(row):
    num_pur = []
    des_tit = row['des + title'].lower()

    for pur in land_purpose_dict.keys():
        search_for_pur = re.search(pur, des_tit)
        if search_for_pur:
            if pur == 'Đất ở':
                new_pat = r'(?:gần|ra|giáp|sát|ngay|đối diện|phù hợp làm|phù hợp lên|chuyển đổi thành)\s' + rf'(?:{pur})'
            else:
                new_pat = r'(?:gần|ra|giáp|sát|ngay|đối diện|phù hợp làm|phù hợp lên|chuyển đổi thành)\s' + rf'(?:{pur.replace("đất ", "")})'
            if re.search(new_pat, des_tit):
                continue
            else:
                num_pur.append(land_purpose_dict[pur])

    if len(num_pur) > 1:
        return 'Mục đích sử dụng khác' # f'Đất hỗn hợp: {num_pur}'
    if len(num_pur) == 1:
        if num_pur[0] == 'Đất ở':
            if re.search('đất ở đô thị|\Wodt\W', des_tit):
                return 'Đất ở đô thị'
            if re.search('Phường', row['short_address']):
                return 'Đất ở đô thị'
        return f'{num_pur[0]}'
    return 'Đất ở'

# land['Mục đích SDD'] = land['des + title'].apply(search_purpose)
land['Mục đích SDD'] = land['des + title'].apply(search_purpose)
land['New Mục đích SDD'] = land.apply(new_search_purpose, axis = 1)
print(land['New Mục đích SDD'].value_counts())
# real không có từ gì: 1, 5, 10, 12, 13, 15, 18, 19, 20, 26, 32, 33, 36, 41, 42, 44, 45, 46, 49, 50, 53, 60, 63, 64, 66, 67, 68, 70, 72, 75, 79, 81, 85, 86, 87, 89, 90, 91, 93, 94, 95, 98, 100, 101, 106, 108, 109, 114, 115, 124, 129, 130, 132, 133, 134, 143, 144, 147, 148, 150, 152, 154, 156, 157, 158, 159, 161, 162, 163, 165, 168, 169, 170, 171, 174, 177, 178, 181, 183, 184, 185, 189, 192 (bán đất mặt tiền đường thương mại), 193, 194, 196, 200, 201, 203, 205, 206, 207, 208, 209, 211

New Mục đích SDD
Đất ở                      1624
Đất ở đô thị                239
Mục đích sử dụng khác        57
Đất thương mại, dịch vụ      21
Đất vườn                     15
Đất trồng cây lâu năm        14
Đất trồng cây hàng năm        6
Đất rừng                      3
Đất khu công nghiệp           2
Name: count, dtype: int64
